In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.rcParams['figure.figsize'] = 9, 6
%load_ext autoreload
%autoreload 2
import sample_data

np.random.seed(0)
N_train = 2 ** 10; N_test = 0
N = N_train + N_test
train = np.arange(N_train); test = np.arange(N_test) + N_train
n_min = 16; n_max = n_min

X, w = sample_data.periodic(N, n_min, n_max, t_max=2*np.pi, even=True,
                            A_shape=1., noise_sigma=1e-0, w_min=0.1, w_max=1.)
M = X[:, :, 1:2]
print(M.shape)

In [ ]:
%%time
import pywt
from scipy.fftpack import fft, dct

Y = dct(M)[:, :, :]
#Y = np.array([pywt.wavedec(m, pywt.Wavelet('db4'))
#              for m in M]).reshape(M.shape)
#A = np.random.normal(size=n_min ** 2).reshape((n_min, n_min))
#Y = (M[:, :, 0] @ A)

print(M.shape, Y.shape)
Y = Y.reshape(M.shape)

In [ ]:
fig, ax = plt.subplots(2, 3, sharey=False)
for i in range(3):
    ax[0, i].plot(X[i][:, 0], M[i, :, 0], 'o')
    ax[0, i].set(xlabel="t", ylabel="y")
    ax[1, i].plot(X[i][:, 0], Y[i, :], 'o')
    ax[1, i].set(xlabel="t", ylabel="y")

In [ ]:
from keras.callbacks import Callback
from IPython.core.display import clear_output

class ClearOutput(Callback):
    def on_epoch_end(self, epoch, logs={}):
        clear_output(wait=True)

In [ ]:
import os
import tensorflow as tf
from keras import backend as K
from keras.layers import (Input, Dense, TimeDistributed, Activation,
                          LSTM, Dropout, merge, Reshape, Flatten,
                          RepeatVector, ActivityRegularization,
                          Convolution1D, MaxPooling1D, Lambda)
from keras.models import Model, Sequential
from keras.optimizers import Adam, SGD
from keras.regularizers import l1, activity_l1
from keras import metrics
from keras.callbacks import ProgbarLogger, TensorBoard
from IPython.display import clear_output, display

sess = tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(
            per_process_gpu_memory_fraction=0.2)))
K.set_session(sess)

#lstm_size = 128
#num_layers = 4
#lr = 4e-3
#drop_frac = 0.2
#run = "lstm_{:03d}_{}_{:1.0e}_drop{}".format(lstm_size, num_layers, lr, drop_frac).replace('e-', 'm').replace('.', '')
#print(run)
#model = Sequential()
#model.add(LSTM(lstm_size, input_shape=(n_max, 1), return_sequences=True))
#for i in range(1, num_layers):
#    model.add(Dropout(drop_frac))
#    model.add(LSTM(lstm_size, return_sequences=(i != num_layers - 1)))
#model.add(Dense(Y.shape[1], activation='linear'))
#adam = Adam(lr=lr)
#model.compile(optimizer=adam, loss='mse', metrics=[])

#model = Sequential()
#model.add(Convolution1D(conv_size, length, border_mode='same', input_shape=(n_max, 1)))
#model.add(MaxPooling1D())
#model.add(Flatten())
#model.add(Dense(Y.shape[1], activation='linear'))
#adam = Adam(lr=lr)
#model.compile(optimizer=adam, loss='mse', metrics=[])
#run = "conv_max_{}_{}_{:1.0e}".format(conv_size, length, lr).replace('e-', 'm')
#print(run)

# Zero out odd/even components
zero_odd = Lambda(lambda x: x * np.tile([0, 1], int(x.get_shape()[-1].value / 2)))
zero_even = Lambda(lambda x: x * np.tile([1, 0], int(x.get_shape()[-1].value / 2)))
# Pass everything through
#zero_odd = Lambda(lambda x: x * np.tile([1, 1], int(x.get_shape()[-1].value / 2)))
#zero_even = Lambda(lambda x: x * np.tile([1, 1], int(x.get_shape()[-1].value / 2)))

dense_size = n_max
lr = 8e-2
rho = 0.0
input = Input(shape=(n_max,))
dense_o = Dense(dense_size / 2, bias=False, W_regularizer=l1(rho))(zero_odd(input))
dense_e = Dense(dense_size / 2, bias=False, W_regularizer=l1(rho))(zero_even(input))

dense_oo = Dense(dense_size / 4, bias=False)(zero_odd(dense_o))
dense_oe = Dense(dense_size / 4, bias=False)(zero_odd(dense_e))
dense_eo = Dense(dense_size / 4, bias=False)(zero_even(dense_o))
dense_ee = Dense(dense_size / 4, bias=False)(zero_even(dense_e))

dense_ooo = Dense(dense_size / 8, bias=False)(zero_odd(dense_oo))
dense_ooe = Dense(dense_size / 8, bias=False)(zero_odd(dense_oe))
dense_oeo = Dense(dense_size / 8, bias=False)(zero_odd(dense_eo))
dense_oee = Dense(dense_size / 8, bias=False)(zero_odd(dense_ee))
dense_eoo = Dense(dense_size / 8, bias=False)(zero_even(dense_oo))
dense_eoe = Dense(dense_size / 8, bias=False)(zero_even(dense_oe))
dense_eeo = Dense(dense_size / 8, bias=False)(zero_even(dense_eo))
dense_eee = Dense(dense_size / 8, bias=False)(zero_even(dense_ee))

merged_1 = merge([dense_o, dense_e],
#merged_1 = merge([dense_ooo, dense_ooe, dense_oeo, dense_oee,
#                  dense_eoo, dense_eoe, dense_eeo, dense_eee], 
                 mode='concat')
output = Dense(n_max, bias=False)(merged_1)
model = Model(input, output)
sgd = SGD(lr=lr)
model.compile(optimizer=sgd, loss='mse', metrics=[metrics.mean_squared_error])
run = "split_{}_{:1.0e}".format(dense_size, lr).replace('e-', 'm')
print(run)

log_dir = os.path.expanduser('~/Dropbox/Documents/timeflow/keras_logs/fourier/{}'.format(run))
!rm -rf $log_dir
history = model.fit(M[train, :, 0], Y[train, :, 0],
                     nb_epoch=5000, batch_size=N_train, validation_split=0.0,
                     callbacks=[TensorBoard(log_dir=log_dir, write_graph=False),
                                ProgbarLogger(), ClearOutput(),
                               ],
#                    verbose=0
                   )
model.save_weights(os.path.join(log_dir, 'raw.h5'), overwrite=True)

In [ ]:
from keras.utils.visualize_util import model_to_dot
from IPython.display import SVG

model_dot = model_to_dot(model).create(prog='dot', format='svg')
print('{} MB'.format(model.count_params() * np.dtype('float32').itemsize / 1e6))
print(model.summary())
SVG(model_dot)

In [ ]:
W = model.get_weights()
[np.mean(np.abs(W[i]) >= 1e-3) for i in range(len(W))]